In [4]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_FRACDIFF, ALL_RAW_FEAT, SSMContainer
from jesse.helpers import date_to_timestamp
import lightgbm as lgb
import pandas as pd
import numpy as np

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles("2025-01-01", "2025-10-01")

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp("2025-03-01")) &
    (df_feat_full.index.to_numpy() < date_to_timestamp("2025-09-30"))
]

getting raw features...
df_raw_feats.shape = (11957, 12872)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [21]:
import json

MODEL_TYPE = "c"
LAG = 7
PRED_NEXT = 1
THRESHOLD = 0.5
MODEL_NAME = f"{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}"

with open("strategies/BinanceBtcDeapV1Voting/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

close_price = candles[:, 2][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")
assert len(datetimes) == len(close_price)

model = lgb.Booster(model_file=f"strategies/BinanceBtcDeapV1Voting/models/model_{MODEL_NAME}.txt")
pred = model.predict(df_selected_feats)
assert len(pred) == len(datetimes)

In [22]:
from research.plot import analyze_confidence_slices

slice_res = analyze_confidence_slices(
    datetimes,
    pred,
    close_price,
    PRED_NEXT,
    granularity=0.1,
    lower_bound=0,
    upper_bound=1,
    threshold=0.5,
)


📊 时序对齐信息:
  - pred_next: 1 (预测未来第1根K线)
  - 原始数据长度: 8067
  - 对齐后数据长度: 8066
  - 丢弃的尾部数据: 1 条

已生成: slice_0.9000_1.0000_ratio_0.3045.jpg (切片区间: [0.9000, 1.0000] (做多), 样本占比: 30.45%)
已生成: slice_0.8000_0.9000_ratio_0.0787.jpg (切片区间: [0.8000, 0.9000] (做多), 样本占比: 7.87%)
已生成: slice_0.7000_0.8000_ratio_0.0485.jpg (切片区间: [0.7000, 0.8000] (做多), 样本占比: 4.85%)
已生成: slice_0.6000_0.7000_ratio_0.0392.jpg (切片区间: [0.6000, 0.7000] (做多), 样本占比: 3.92%)
已生成: slice_0.5000_0.6000_ratio_0.0333.jpg (切片区间: [0.5000, 0.6000] (做多), 样本占比: 3.33%)
已生成: slice_0.4000_0.5000_ratio_0.0403.jpg (切片区间: [0.4000, 0.5000] (做空), 样本占比: 4.03%)
已生成: slice_0.3000_0.4000_ratio_0.0415.jpg (切片区间: [0.3000, 0.4000] (做空), 样本占比: 4.15%)
已生成: slice_0.2000_0.3000_ratio_0.0475.jpg (切片区间: [0.2000, 0.3000] (做空), 样本占比: 4.75%)
已生成: slice_0.1000_0.2000_ratio_0.0724.jpg (切片区间: [0.1000, 0.2000] (做空), 样本占比: 7.24%)
已生成: slice_0.0000_0.1000_ratio_0.2941.jpg (切片区间: [0.0000, 0.1000] (做空), 样本占比: 29.41%)


In [2]:
from strategies.BinanceBtcDeapV1Voting.models.config import LGBMContainer

model_container = LGBMContainer(
    model_type=MODEL_TYPE,
    lag=LAG,
    pred_next=PRED_NEXT,
    threshold=THRESHOLD,
)

In [3]:
model_container.save_filters()

Filters saved to /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/model_c_L5_N1_filters.json


In [ ]:
ft = 0.01   ### 0.1, 0.05, 0.01

if ft == 0.1:
    da = 1
    xiao = 0.9
    z = 5

if ft == 0.05:
    da = 1
    xiao = 0.95
    z = 10
    
if ft == 0.01:
    da = 1
    xiao = 0.99
    z = 50


for i in range(z):        

    data_1['final'] = 0

    mask = (data_1['score'] < da) & (data_1['score'] >= xiao)
    data_1['final'] = np.where(mask, data_1['volume'], 0)
    
    true_count = mask.sum() / data_1_size 
    
    ### 更新置信度
    da = round((da - ft), 2)
    xiao = round((xiao - ft), 2)

    data_1['high'] = data_1['final'].cumsum()
    data_1['open'] = rrr * data_1['high'] / m

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1['eob'], data_1['open'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))   
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(da) + "_" + str(true_count.round(2)) + ".jpg")   
    plt.close()

In [25]:
df_model = pd.read_csv("model_search_results.csv")
df_model[
    (df_model['log_return_lag'] == 10) &
    (df_model['pred_next'] == 1) &
    (df_model['model_type'] == ("classifier" if MODEL_TYPE == "c" else "regressor"))
].iloc[0]

log_return_lag                                                      10
pred_next                                                            1
model_type                                                  classifier
best_score                                                    0.896588
feature_count                                                      150
timestamp                                          2025-10-03 10:17:46
status                                                       completed
duration_seconds                                           3503.320826
best_params          {"objective": "binary", "num_threads": -1, "ve...
selected_features    ["cmma", "fisher", "cwt_w32_1_lag5", "williams...
Name: 37, dtype: object